In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow
%pip install pyautogui

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emres/turkish-deasciifier.git to c:\users\cemke\appdata\local\temp\pip-req-build-sw3do5l9
  Resolved https://github.com/emres/turkish-deasciifier.git to commit 665154c734b09485c3d11ce0038cd121dd109594
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may 

  Running command git clone --filter=blob:none --quiet https://github.com/emres/turkish-deasciifier.git 'C:\Users\cemke\AppData\Local\Temp\pip-req-build-sw3do5l9'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import pyautogui
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as st
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline
from transformers import pipeline
from trnlp import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

2023-01-14 04:14:34,541 - numexpr.utils - INFO
Msg: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.

2023-01-14 04:14:34,542 - numexpr.utils - INFO
Msg: NumExpr defaulting to 8 threads.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()
            
            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Degistirilmis'] = [tweet.full_text for tweet in tweets]
            #DataSet['Tweet_Ceviri']=[tweet.full_text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            DataSet.to_csv('tw.csv',index=False)
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,arananTweet):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=arananTweet,result_type="new", lang='tr',tweet_mode="extended")
            results=[]
            for item in cursor.items(20):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)            
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Dataset=pd.read_csv('tw.csv',encoding="utf-8")                   
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.WordSeparation(DataSet) 
            getTweets.ToxicSentence(DataSet) 
            getTweets.TrWordSeparation(DataSet)
            getTweets.TextblobSentiment(DataSet)           
            DataSet.to_csv('tw.csv',index=False)

    def TextblobSentiment(DataSet):
        try:
            data=DataSet
            print ("Duygu Analizi Yapılıyor...")            
            data["Tür"] = data["Tweet_Ayrı"].apply(getTweets.getPolarity)
            data["Yoğunluk"] = data["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            data.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def TrWordSeparation(DataSet):
        try:
            data=DataSet
            print("Türkçe Kelime Ayrıştırma Yapılıyor...")
            data["Tweet_Ayrı_TR"]=data["Tweet_Degistirilmis"]
            for index in range(len(data["Tweet_Ayrı_TR"])):
                data["Tweet_Ayrı_TR"][index] = simple_token(data["Tweet_Ayrı_TR"][index], sw=stopwords)
            print("Türkçe Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            data=DataSet
            print ("Tweetler Temizleniyor...")
            data["Tweet_Degistirilmis"]=data["Tweet"]
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-ZğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|RT[\s]+|amp[\s]+|gt;')
            for index in range(len(data["Tweet_Degistirilmis"])):        
                data["Tweet_Degistirilmis"][index]=pattern.sub('', data["Tweet_Degistirilmis"][index])    
                data["Tweet_Degistirilmis"][index]=data["Tweet_Degistirilmis"][index].lower()              
            data["Tweet_Degistirilmis"]=data["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))  
                              
    def TweetTranslate(DataSet):
        try:
            data=DataSet
            print ("Tweetler Çeviriliyor...")
            data["Tweet_Ceviri"]=data["Tweet_Degistirilmis"]
            for index in range(len(data["Tweet_Ceviri"])):
                Ceviri_Blob=TextBlob(data["Tweet_Ceviri"][index])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                data["Tweet_Ceviri"][index]=Eng_Blob  
                data["Tweet_Ceviri"][index]=data["Tweet_Ceviri"][index].lower()
            print ("Tweetler Çevirildi!")                      
            return data      
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def SentenceNormalization(DataSet):
        try:
            data=DataSet
            print("Cümle Normalizasyonu Yapılıyor...")
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)
            for index in range(len(data["Tweet_Degistirilmis"])):                
                data["Tweet_Degistirilmis"][index]= normalizer.normalize(data["Tweet_Degistirilmis"][index].lower())
            print("Cümle Normalizasyonu Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def WordSeparation(DataSet):
        try:
            data=DataSet
            print("İngilizce Kelime Ayrıştırma Yapılıyor...")
            additional  = ['sq','gt']
            stop = set().union(st.words('english'),additional)
            
            data["Tweet_Ayrı"] = data["Tweet_Ceviri"].str.split()
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].str.join(" ")
            print("İngilizce Kelime Ayrıştırma Yapıldı!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
    
    def ToxicSentence(DataSet):
        try:
            data=DataSet
            print("Toxic Cümle Analizi Yapılıyor...")
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            data["Tweet_Toxic"]=data["Tweet_Ceviri"]
            for index in range(len(data["Tweet_Toxic"])):
                sonuc=pipeline(str(data["Tweet_Toxic"][index]))
                if sonuc[0]['label']=="toxic":
                    data["Tweet_Toxic"][index]="Toxic"
                else:
                    data["Tweet_Toxic"][index]="Toxic değil"    
            print("Toxic Cümle Analizi Yapıldı!")
            return data  
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "%75 - %100"
            elif sub >=0.5 and sub <0.75:
                return "%50 - %75"
            elif sub >=0.25 and sub <0.50:
                return "%25 - %50"
            else:
                return "0 - 25%"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [85]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
while True:
    #aramaTuru=int(pyautogui.prompt(text="Ne Tür Bir Arama Yapacaksınız? \n 1=Hastag \n 2=Kelime Bazlı Arama \n 3=Kullanıcı", title='Arama Türü' , default='1'))
    aramaTuru=pyautogui.confirm('Ne Tür Bir Arama Yapacaksınız?',title='Arama Türü', buttons=['Hastag','Kelime','Kullanıcı'])
    if aramaTuru=="Hastag":
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=("#"+arananKelime+" -filter:retweets")
        tw.searchTweet(arananTweet=temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    elif aramaTuru=='Kelime':
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=(arananKelime+" -filter:retweets")
        tw.searchTweet(arananTweet=temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    elif aramaTuru=='Kullanıcı':
        arananKelime=pyautogui.prompt(text="Aranacak Kelimeyi Girin:", title='Aranacak Kelime' , default='python')
        temp=("@"+arananKelime+" -filter:retweets")
        tw.searchTweet(temp)
        secim=pyautogui.confirm(text='Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break
    else:
        secim=pyautogui.confirm(text='Hatalı Seçim Yaptınız Tekrar Denemek İster Misiniz?', title='Hata', buttons=["Tekrar","İptal"])
        if secim=="Tekrar":
            continue
        else:
            break


Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Cümle Normalizasyonu Yapılıyor...
2023-01-14 05:20:13,749 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 8.495999336242676

Cümle Normalizasyonu Yapıldı!
Tweetler Çeviriliyor...
Tweetler Çevirildi!
İngilizce Kelime Ayrıştırma Yapılıyor...
İngilizce Kelime Ayrıştırma Yapıldı!
Toxic Cümle Analizi Yapılıyor...
Toxic Cümle Analizi Yapıldı!
Türkçe Kelime Ayrıştırma Yapılıyor...
Türkçe Kelime Ayrıştırma Yapıldı!
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [86]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head(10)

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tweet_Toxic,Tweet_Ayrı_TR,Tür,Yoğunluk
0,Üniversite İstatistik Ders & Tez-Ödev Danışmanlık,2023-01-13 22:53:26+00:00,"Tüm sınava girecek arkadaşlara ""kolaylıklar dileriz"" \n#İstatistikSınavı #Olasılık #MATLAB #SPSS #LISREL #STATA #Minitab #SAS #R #Python #DataScience #İstatistikselAnaliz #YolAnalizi #YEM #TezAnalizi #Proje #finalhaftası #Final #Finals",tüm sınava girecek arkadaşlara kolaylıklar dileriz,we wish all the friends who will take all the exams.,wish friends take exams.,Toxic değil,"['sınava', 'girecek', 'arkadaşlara', 'kolaylıklar', 'dileriz']",Nötr,0 - 25%
1,"new String(""Özgür"")",2023-01-13 21:54:09+00:00,Ya aranızda adam akıllı bir keyboard key'lerini senkron şekilde işleyen python paketi bilen var mı deliricem aqqqqq,ya aranızda adam akıllı bir keyboard köylerini senkron şekilde işleyen python paketi bilen var mı delireceğim aqqqqq,"or do anyone know the python package that syncs a smart keyboard villages between you, aqqqqq","anyone know python package syncs smart keyboard villages you, aqqqqq",Toxic değil,"['aranızda', 'adam', 'akıllı', 'keyboard', 'köylerini', 'senkron', 'şekilde', 'işleyen', 'python', 'paketi', 'bilen', 'delireceğim', 'aqqqqq']",Pozitif,%50 - %75
2,future,2023-01-13 20:59:10+00:00,@Ateist_TV @finance_python aslında bu Afganistan 🇦🇫 yaşayan Halk demeki böyle salaş yaşamayı seviyor,aslında bu afganistan yaşayan halk demek böyle salaş yaşamayı seviyor,"in fact, this people who live in afghanistan like to live such shabby","fact, people live afghanistan like live shabby",Toxic değil,"['afganistan', 'yaşayan', 'halk', 'salaş', 'yaşamayı', 'seviyor']",Pozitif,%50 - %75
3,Mlhkrtss,2023-01-13 20:55:41+00:00,@hippomanick Aslında bir ara öğrenecektim. Volkan Taşcı Python kursu ve kitabını almıştım. Pycharm bile kurmuştum.,aslında bir ara öğrenecektim volkan tasçı python kursu ve kitabını almıştım pycharm bile kurmuştum,"actually, i was going to learn for a while, i bought the volkan tasçı python course and my book i even established a pycharm","actually, going learn while, bought volkan tasçı python course book even established pycharm",Toxic değil,"['ara', 'öğrenecektim', 'volkan', 'tasçı', 'python', 'kursu', 'kitabını', 'almıştım', 'pycharm', 'kurmuştum']",Nötr,0 - 25%
4,julius,2023-01-13 20:49:27+00:00,Sabahtan beri boktan bir python kütüphanesini yüklemeye çalışıyorum onsuz kodum çalışmıyor çünkü kafayı yicem,sabahtan beri boktan bir python kütüphanesini yüklemeye çalışıyorum onsuz kodum çalışmıyor çünkü kafayı biçem,"i've been trying to install a shitty python library since morning, my code without it is not working because","i've trying install shitty python library since morning, code without working",Toxic değil,"['sabahtan', 'beri', 'boktan', 'python', 'kütüphanesini', 'yüklemeye', 'çalışıyorum', 'onsuz', 'kodum', 'çalışmıyor', 'kafayı', 'biçem']",Nötr,0 - 25%
5,Mlhkrtss,2023-01-13 20:47:58+00:00,@hippomanick Öğlenki mesajımın bir tanesini yanlış anlamayın lütfen. Bu yıl öğreneceğim Python. O da siz ve sevgilinizin sayesinde. A takımına veya Python'un avcıları ekibine ihtiyacım olacak. 😀 Ama zamanlama sorunu var. Önce Tailwind ve Bootstrap arasında kaldım.,öğlenki mesajımın bir tanesini yanlış anlamayın lütfen bu yıl öğreneceğim python o da siz ve sevgilinizin sayesinde a takımına veya pythonun avcıları ekibine ihtiyacım olacak ama zamanlama sorunu var önce tailwind ve bootstrap arasında kaldım,"please do not misunderstand one of my noon message, please i will learn this year, python, and thanks to you and your lover, i will need a team a or python's hunters team, but i have a timing problem before i stayed between tailwind and bootstrap","please misunderstand one noon message, please learn year, python, thanks lover, need team python's hunters team, timing problem stayed tailwind bootstrap",Toxic değil,"['öğlenki', 'mesajımın', 'tanesini', 'yanlış', 'anlamayın', 'lütfen', 'yıl', 'öğreneceğim', 'python', 'o', 'sevg

In [87]:
Veri.tail(10)

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tweet_Toxic,Tweet_Ayrı_TR,Tür,Yoğunluk
10,chapter 0: WITH 👍,2023-01-13 18:09:36+00:00,"@JAESTANSV Python ile gördük bu dersi. Ama daha çok java ile gösteriliyor, biz sınıfça Python dilinde görmek istedik :)",python ile gördük bu dersi ama daha çok java ile gösteriliyor biz sınıfça python dilinde görmek istedik,"we saw this lesson with python, but it is more shown with java, we wanted to see it in python language.","saw lesson python, shown java, wanted see python language.",Toxic değil,"['python', 'gördük', 'dersi', 'java', 'gösteriliyor', 'sınıfça', 'python', 'dilinde', 'görmek', 'istedik']",Nötr,0 - 25%
11,☯,2023-01-13 17:48:27+00:00,@umutyerebakmaz @purkinjeey İşe alımlarda bu dediğiniz sıkıntı olmaz mı? Bir çok iş ilanı gezindim bir çoğu php ve python üzerine.,işe alımlarda bu dediğiniz sıkıntı olmaz mı bir çok iş ilanı gezindim bir çoğu php ve python üzerine,wouldn't it be a problem in recruitment?,problem recruitment?,Toxic değil,"['işe', 'alımlarda', 'dediğiniz', 'sıkıntı', 'olmaz', 'iş', 'ilanı', 'gezindim', 'php', 'python', 'üzerine']",Nötr,0 - 25%
12,Eflatun,2023-01-13 17:34:47+00:00,"Tonlarca betonu bir çırpıda zıkkımlanan, ayrıca bass baritone olduğu şüphe götürmez Akhenaton’un melankolik teşrifatçı, kırmızımtrak Python’u arkadan geçen Alfred Hitchcock’un hiçyoktan 1 saniyesi mi diye sakın ola içinden bile geçirme o sadece su verilmeyen çeliğin atonal müziği https://t.co/6AgvVaumsj",tonlarca betonu bir çırpıda zıkkımlanan ayrıca bass bariton olduğu şüphe götürmez akhenatonun melankolik teşrifatçı kırmızımtırak pythonu arkadan geçen alfred hitchcockun hiç yoktan saniyesi mı diye sakın ola içinden bile geçirme o sadece su verilmeyen çeliğin atonal müziği,"tons of concrete in a snap in a snap of bass baritone in the doubt of the melancholic encouragement of the melancholic graceful pythoni alfred hitchcock of the rear of the alfred hitchcock's never to pass through it, even the water of the steel of the steel of the steel","tons concrete snap snap bass baritone doubt melancholic encouragement melancholic graceful pythoni alfred hitchcock rear alfred hitchcock's never pass it, even water steel steel steel",Toxic değil,"['tonlarca', 'betonu', 'çırpıda', 'zıkkımlanan', 'ayrıca', 'bass', 'bariton', 'olduğu', 'şüphe', 'götürmez', 'akhenatonun', 'melankolik', 'teşrifatçı', 'kırmızımtırak', 'pythonu', 'arkadan', 'geçen', 'alfred', 'hitchcockun', 'yoktan', 'saniyesi', 'sakın', 'ola', 'içinden', 'geçirme', 'o', 'sadece', 'su', 'verilmeyen', 'çeliğin', 'atonal', 'müziği']",Nötr,%25 - %50
13,Dünya'nın Verisi,2023-01-13 15:55:55+00:00,TIOBE Endeksi’ne Göre En Popüler İlk 50 Programa Dili (2023 Ocak):https://t.co/8900VwqBZ5\n\n✔️#Python lider\n✔️Çocuklar için önerilen #Scratch ilk 20'de👏 https://t.co/HVfKiJGhdH,töbe endeksine göre en popüler ilk programa dili ocak lidercocuklar için önerilen ilk de,"according to the repentance index, the most popular first program is the first recommended for the children of january lie.","according repentance index, popular first program first recommended children january lie.",Toxic değil,"['töbe', 'endeksine', 'göre', 'popüler', 'ilk', 'programa', 'dili', 'ocak', 'lidercocuklar', 'önerilen', 'ilk']",Pozitif,%50 - %75
14,Anlaşılır Ekonomi💸💸💸,2023-01-13 15:44:56+00:00,"Python ile Makine Öğrenmesi yeni video yayında.\n\nNLP-9\n\nWordCloud Nedir?\n\nİyi seyirler, iyi tatiller efendim...\n\nhttps://t.co/Sif5CQyBYB https://t.co/NA7d8q2Fxd",python ile makine öğrenmesi yeni video yayındanlpwordcloud nediriyi seyirler iyi tatiller efendim,"machine learning with python new video broadcasting what is a good holidays, sir","machine learning python new video broadcasting good holidays, sir",Toxic değil,"['python', 'makine', 'öğrenmesi', 'yeni', 'video', 'yayındanlpwordcloud', 'nediriyi', 'seyirler', 'iyi', 'tatiller', 'efendim']",Pozitif,%50 - %75
15,python.finance,2023-01-13 14:09:34+00:00,"Avrupalıyla farkımız\nOnlar acılarından,

In [88]:
#Eksik Cümle
unmasker=pipeline('fill-mask', model='xlm-roberta-base')
